# Advent of Code #


In [192]:
# set up the environment
import numpy as np
import re


 ## Day 5: Hydrothermal Venture ##

 ### Part 1 ###

You come across a field of hydrothermal vents on the ocean floor! These vents constantly produce large, opaque clouds, so it would be best to avoid them if possible.

They tend to form in lines; the submarine helpfully produces a list of nearby lines of vents (your puzzle input) for you to review. For example:
```
0,9 -> 5,9
8,0 -> 0,8
9,4 -> 3,4
2,2 -> 2,1
7,0 -> 7,4
6,4 -> 2,0
0,9 -> 2,9
3,4 -> 1,4
0,0 -> 8,8
5,5 -> 8,2
```
Each line of vents is given as a line segment in the format x1,y1 -> x2,y2 where x1,y1 are the coordinates of one end the line segment and x2,y2 are the coordinates of the other end. These line segments include the points at both ends. In other words:

    An entry like 1,1 -> 1,3 covers points 1,1, 1,2, and 1,3.
    An entry like 9,7 -> 7,7 covers points 9,7, 8,7, and 7,7.

For now, only consider horizontal and vertical lines: lines where either x1 = x2 or y1 = y2.

So, the horizontal and vertical lines from the above list would produce the following diagram:
```
.......1..
..1....1..
..1....1..
.......1..
.112111211
..........
..........
..........
..........
222111....
```
In this diagram, the top left corner is 0,0 and the bottom right corner is 9,9. Each position is shown as the number of lines which cover that point or . if no line covers that point. The top-left pair of 1s, for example, comes from 2,2 -> 2,1; the very bottom row is formed by the overlapping lines 0,9 -> 5,9 and 0,9 -> 2,9.

To avoid the most dangerous areas, you need to determine the number of points where at least two lines overlap. In the above example, this is anywhere in the diagram with a 2 or larger - a total of 5 points.

Consider only horizontal and vertical lines. At how many points do at least two lines overlap?

### Import the data ###

In [193]:
raw_vent_data = np.genfromtxt("data/vents.dat", dtype="str", delimiter=" -> ")
# raw_vent_data = np.genfromtxt("data/vent_test.dat", dtype="str", delimiter=" -> ")
map_size = [1000, 1000]

# define a vent class
class Vent:
    def __init__(self) -> None:
        self.co_ords = np.empty((2, 2), dtype=int)
        self.vertical = False
        self.horizontal = False

    def __str__(self) -> str:
        return f"{self.co_ords[:]}\n Horizontal: {self.horizontal}, Vertical: {self.vertical}"

    def create(self, raw_co_ords: str) -> None:
        co_ord_str = f"{raw_co_ords[0]},{raw_co_ords[1]}".split(",")
        self.co_ords = np.array(list(map(int, co_ord_str))).reshape(2, 2)
        if self.co_ords[0][0] == self.co_ords[1][0]:
            self.vertical = True
        if self.co_ords[0][1] == self.co_ords[1][1]:
            self.horizontal = True

### Set up the Vents ###

In [194]:
def set_vents() -> np.ndarray:
    vents = []
    for raw_vent in raw_vent_data:
        vent = Vent()
        vent.create(raw_vent)
        vents.append(vent)
    return vents

### Create the vent map ###

In [195]:
orthogonal_vent_map = np.zeros(map_size, dtype=int)

### Mark horizontal and vetical lines on the vent_map ###

Make allowances for the lines to be in any direction!

In [196]:
vents = set_vents()
for vent in vents:
    step = 1
    if vent.horizontal:
        y = vent.co_ords[0, 1]
        x1 = vent.co_ords[0, 0]
        x2 = vent.co_ords[1, 0]
        if x1 > x2:
            step = -1
        for x in range(x1, x2+step, step):
            orthogonal_vent_map[y, x] += 1
    if vent.vertical:
        x = vent.co_ords[0,0]
        y1 = vent.co_ords[0,1]
        y2 = vent.co_ords[1,1]
        if y1 > y2:
            step = -1
        for y in range(y1, y2+step, step):
            orthogonal_vent_map[y, x] += 1

### Count the number of entries greater than 1 ###

In [197]:
orthogonal_count = np.count_nonzero(orthogonal_vent_map > 1)
print(f'Number of orthogonal danger points {orthogonal_count}')

Number of orthogonal danger points 6856


### Part Two ###

Unfortunately, considering only horizontal and vertical lines doesn't give you the full picture; you need to also consider diagonal lines.

Because of the limits of the hydrothermal vent mapping system, the lines in your list will only ever be horizontal, vertical, or a diagonal line at exactly 45 degrees. In other words:

    An entry like 1,1 -> 3,3 covers points 1,1, 2,2, and 3,3.
    An entry like 9,7 -> 7,9 covers points 9,7, 8,8, and 7,9.

Considering all lines from the above example would now produce the following diagram:
```
1.1....11.
.111...2..
..2.1.111.
...1.2.2..
.112313211
...1.2....
..1...1...
.1.....1..
1.......1.
222111....
```
You still need to determine the number of points where at least two lines overlap. In the above example, this is still anywhere in the diagram with a 2 or larger - now a total of 12 points.

### Add the diagonal lines to the map ###
Ignore vertical and horizontal lines as they have already been dealt with.  
Also, normalise the co-ordinates before plotting

In [198]:
diagonal_vent_map = np.zeros(map_size, dtype=int)
vents = set_vents()
for x, vent in enumerate(vents):
    if vent.horizontal or vent.vertical:
        continue
    x1 = vent.co_ords[0,0]
    x2 = vent.co_ords[1,0]
    y = vent.co_ords[0,1]
    y2 = vent.co_ords[1,1]
    x_step = 1 if x2 > x1 else -1
    y_step = 1 if y2 > y else -1
    for x in range(x1, x2+x_step, x_step):
            diagonal_vent_map[y,x] += 1
            y += y_step

### Count the revised entries greater than 2 ###

In [199]:
diagonal_count = np.count_nonzero(diagonal_vent_map > 1)
print(f'Number of diagonal danger points {diagonal_count}')

Number of diagonal danger points 5721


### Add the two maps together  and get new total of danger points ###

In [200]:
full_vent_map = orthogonal_vent_map + diagonal_vent_map
full_count = np.count_nonzero(full_vent_map > 1)
print(f'Number of full danger points {full_count}')

Number of full danger points 20666
